In [57]:
import sklearn as sk
import statsmodels.api as sm
from statsmodels.formula.api import glm
from sklearn import preprocessing
from sklearn import decomposition
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sdv
import pickle as pk
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay, roc_auc_score, confusion_matrix
from sklearn.metrics import RocCurveDisplay, roc_auc_score, confusion_matrix
import imblearn
from sklearn import metrics
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

In [75]:
df = pd.read_csv('./train_combined_Species.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df = df.fillna(0).drop_duplicates()

def drop_almost_zero(df, percentage):
    column_cut_off = int(percentage/100*len(df))
    b = (df == 0).sum(axis='rows')
    df = df[ b[ b <= column_cut_off].index.values ]
    return df
# df = drop_almost_zero(df,90)

inputvalues = df.drop(['label'], axis=1)
outputvalues = df['label']
X, y = inputvalues.values, outputvalues.values
X = X.astype('float32')

scaler = MinMaxScaler().fit(X)
pk.dump(scaler, open("scaler.pkl","wb"))
X = scaler.transform(X)

pca = PCA(n_components=90).fit(X)
pk.dump(pca, open("pca.pkl","wb"))
X = pca.transform(X)

In [77]:
from imblearn.under_sampling import RandomUnderSampler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
sampler = RandomOverSampler(sampling_strategy='all')
X_train, y_train = sampler.fit_resample(X_train, y_train)

clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Classification Report: \n {}\n".format(metrics.classification_report(y_test, y_pred)))
print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, y_pred)))

Classification Report: 
               precision    recall  f1-score   support

           0       0.45      0.60      0.51        85
           1       0.57      0.25      0.35        93
           2       0.00      0.00      0.00        12
           3       0.07      0.12      0.09         8
           4       0.18      0.50      0.27         4

    accuracy                           0.38       202
   macro avg       0.26      0.29      0.24       202
weighted avg       0.46      0.38      0.38       202


Confusion Matrix: 
 [[51 14 10  5  5]
 [50 23 12  6  2]
 [ 8  1  0  1  2]
 [ 4  2  1  1  0]
 [ 1  0  0  1  2]]



In [70]:
#df["label"].value_counts()
y = y.reshape(-1, 1)
y = OneHotEncoder(categories=[['0', '1', '2', '3', '4']], sparse=False).fit_transform(y)
y

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
sampler = RandomOverSampler(sampling_strategy='all')
X_train, y_train = sampler.fit_resample(X_train, y_train)

In [62]:
clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Classification Report: \n {}\n".format(metrics.classification_report(y_test, y_pred)))
print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, y_pred)))

Classification Report: 
               precision    recall  f1-score   support

           0       0.48      0.59      0.53        85
           1       0.55      0.26      0.35        93
           2       0.05      0.08      0.06        12
           3       0.06      0.12      0.08         8
           4       0.07      0.25      0.11         4

    accuracy                           0.38       202
   macro avg       0.24      0.26      0.23       202
weighted avg       0.46      0.38      0.39       202


Confusion Matrix: 
 [[50 19  7  7  2]
 [40 24 11  8 10]
 [ 8  0  1  2  1]
 [ 5  0  2  1  0]
 [ 1  1  1  0  1]]



In [69]:
clf = RandomForestClassifier(n_estimators=1000, oob_score=True, random_state=42)
clf.fit(X_train_oversampled, y_train_oversampled)  # type: ignore
y_pred = clf.predict(X_test)

'''
feature_names = df.drop(['label'], axis=1).columns
importances = clf.feature_importances_
forest_importances = pd.Series(importances, index = feature_names)
t = forest_importances.sort_values(ascending=False)
print(t)
'''

a = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy of the model based on accuracy of test set: {a: .4f}")
print(f"Error of test set: {1 - a:.4f}")
b = clf.oob_score_
print(f"Out of bag score (samples in training set): {b:.4f}")
print(f"out-of-bag error (samples in training set): {1 - b:.4f}")
print(f"Accuracy of Training set: {clf.score(X_train, y_train) :.4f}")
print("Classification Report: \n {}\n".format(metrics.classification_report(y_test, y_pred)))
print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, y_pred)))

Accuracy of the model based on accuracy of test set:  0.5198
Error of test set: 0.4802
Out of bag score (samples in training set): 0.8083
out-of-bag error (samples in training set): 0.1917
Accuracy of Training set: 1.0000
Classification Report: 
               precision    recall  f1-score   support

           0       0.49      0.62      0.55        85
           1       0.56      0.56      0.56        93
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         8
           4       0.00      0.00      0.00         4

    accuracy                           0.52       202
   macro avg       0.21      0.24      0.22       202
weighted avg       0.46      0.52      0.49       202


Confusion Matrix: 
 [[53 32  0  0  0]
 [41 52  0  0  0]
 [ 8  4  0  0  0]
 [ 6  2  0  0  0]
 [ 1  3  0  0  0]]



C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [ ]:
from sdv.tabular import CTGAN, GaussianCopula

df0 = df.loc[(df['label'] == 0)]
df1 = df.loc[df['label'] == 1]
df01 = pd.concat([df0, df1], axis=0)

model = GaussianCopula()
model.fit(df01)
new_data = model.sample(num_rows=1200)

inputvalues = new_data.drop(['label'], axis=1)
outputvalues = new_data['label']
X_synth, y_synth = inputvalues.values, outputvalues.values
X_synth = X_synth.astype('float32')

In [ ]:
X_train_synth, X_test_synth, y_train_synth, y_test_synth = train_test_split(X_synth, y_synth, test_size=0.2, random_state=1234)
sampler = RandomOverSampler(sampling_strategy='all')
X_train_synth_oversampled, y_train_synth_oversampled = sampler.fit_resample(X_train_synth, y_train_synth)

In [144]:
df0 = df.loc[(df['label'] == 0)]
df1 = df.loc[df['label'] == 4]
df01 = pd.concat([df0, df1], axis=0)

def drop_almost_zero(df, percentage):
    column_cut_off = int(percentage/100*len(df))
    b = (df == 0).sum(axis='rows')
    df = df[ b[ b <= column_cut_off].index.values ]
    return df
#df01 = drop_almost_zero(df01,90)

inputvalues = df01.drop(['label'], axis=1)
outputvalues = df01['label']
X, y = inputvalues.values, outputvalues.values
X = X.astype('float32')

scaler = MinMaxScaler().fit(X)
#pk.dump(scaler, open("scaler.pkl","wb"))
X = scaler.transform(X)

#pca = PCA(n_components=40).fit(X)
#pk.dump(pca, open("pca.pkl","wb"))
#X = pca.transform(X)

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
sampler = RandomOverSampler(sampling_strategy='all')
X_train, y_train = sampler.fit_resample(X_train, y_train)

In [146]:
clf = svm.SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Classification Report: \n {}\n".format(metrics.classification_report(y_test, y_pred)))
print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, y_pred)))

Classification Report: 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98        90
           4       0.00      0.00      0.00         4

    accuracy                           0.96        94
   macro avg       0.48      0.50      0.49        94
weighted avg       0.92      0.96      0.94        94


Confusion Matrix: 
 [[90  0]
 [ 4  0]]



C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,